# Creating Database and Table

In [1]:
import sqlalchemy as db
import pandas as pd

from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Column, String, Integer, Date, Table, ForeignKey, Boolean
from sqlalchemy.orm import relationship, backref

import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


In [2]:
# Creates: a SQLAlchemy Engine that will interact with our sqlite database,
engine = db.create_engine('sqlite:///census.sqlite')

# Creates: a SQLAlchemy ORM session factory bound to this engine
Session = sessionmaker(bind=engine)

# Creates: a base class for our classes definition
Base = declarative_base()


connection = engine.connect()
metadata = db.MetaData()
# census = db.Table('census', metadata, autoload=True, autoload_with=engine)

In [3]:
emp = db.Table('emp', metadata,
              db.Column('Id', db.Integer()),
              db.Column('name', db.String(255), nullable=False),
              db.Column('salary', db.Float(), default=100.0),
              db.Column('active', db.Boolean(), default=True)
              )

metadata.create_all(engine) #Creates the table

In [4]:
# Print the column names
print(emp.columns.keys())

['Id', 'name', 'salary', 'active']


In [5]:
# Print full table metadata
print(repr(metadata.tables['emp']))

Table('emp', MetaData(bind=None), Column('Id', Integer(), table=<emp>), Column('name', String(length=255), table=<emp>, nullable=False), Column('salary', Float(), table=<emp>, default=ColumnDefault(100.0)), Column('active', Boolean(), table=<emp>, default=ColumnDefault(True)), schema=None)


# Inserting Data

In [6]:
#Inserting record one by one
query = db.insert(emp).values(Id=1, name='naveen', salary=60000.00, active=True) 
ResultProxy = connection.execute(query)



In [7]:
#Inserting many records at ones
query = db.insert(emp) 
values_list = [{'Id':'2', 'name':'ram', 'salary':80000, 'active':False},
               {'Id':'3', 'name':'ramesh', 'salary':70000, 'active':True}]
ResultProxy = connection.execute(query,values_list)

In [8]:
results = connection.execute(db.select([emp])).fetchall()
df = pd.DataFrame(results)
df.columns = results[0].keys()
df.head(4)

,Id,name,salary,active
0,1,naveen,60000.0,True
1,2,ram,80000.0,False
2,3,ramesh,70000.0,True
3,1,naveen,60000.0,True


In [9]:
# Print the column names
# print(census.columns.keys())

In [10]:
# Print full table metadata
# print(repr(metadata.tables['census']))

# Querying

In [11]:
#Equivalent to 'SELECT * FROM census'
# query = db.select([census])

In [12]:
# ResultProxy = connection.execute(query)

In [13]:
# ResultSet = ResultProxy.fetchall()

In [14]:
# ResultSet[:3]

# Class: Movie

In [15]:
# Source: https://auth0.com/blog/sqlalchemy-orm-tutorial-for-python-developers/

# As many movies can have many actors and vice-versa, 
# we will need to create a Many To Many relationship between these two classes.

# we created a movies_actors_association table that connects rows of actors and rows of movies
movies_actors_association = Table(
    'movies_actors', Base.metadata,
    Column('movie_id', Integer, ForeignKey('movies.id')),
    Column('actor_id', Integer, ForeignKey('actors.id'))
)


class Movie(Base):
    __tablename__ = 'movies'

    id = Column(Integer, primary_key=True)
    title = Column(String)
    release_date = Column(Date)
    # and we added the actors property to Movie and configured the 
    # movies_actors_association as the intermediary table.
    actors = relationship("Actor", secondary=movies_actors_association)

    def __init__(self, title, release_date):
        self.title = title
        self.release_date = release_date

# Class: Actor

In [16]:
class Actor(Base):
    __tablename__ = 'actors'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    birthday = Column(Date)

    def __init__(self, name, birthday):
        self.name = name
        self.birthday = birthday

# Class: Stuntman

In [17]:
# In this class, we have defined that the actor property references an 
# instance of Actor and that this actor will get a property called stuntman that is not a list (uselist=False). 
# That is, whenever we load an instance of Stuntman, 
# SQLAlchemy will also load and populate the Actor associated with this stuntman.

class Stuntman(Base):
    __tablename__ = 'stuntmen'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    active = Column(Boolean)
    actor_id = Column(Integer, ForeignKey('actors.id'))
    actor = relationship("Actor", backref=backref("stuntman", uselist=False))

    def __init__(self, name, active, actor):
        self.name = name
        self.active = active
        self.actor = actor

# Class: ContactDetails

In [18]:
class ContactDetails(Base):
    __tablename__ = 'contact_details'

    id = Column(Integer, primary_key=True)
    phone_number = Column(String)
    address = Column(String)
    actor_id = Column(Integer, ForeignKey('actors.id'))
    actor = relationship("Actor", backref="contact_details")

    def __init__(self, phone_number, address, actor):
        self.phone_number = phone_number
        self.address = address
        self.actor = actor

# Persisting Data with SQLAlchemy ORM

In [19]:
from datetime import date

# 2 - generate database schema
Base.metadata.create_all(engine)

# 3 - create a new session
session = Session()


In [20]:
# 4 - create movies
bourne_identity = Movie("The Bourne Identity", date(2002, 10, 11))
furious_7 = Movie("Furious 7", date(2015, 4, 2))
pain_and_gain = Movie("Pain & Gain", date(2013, 8, 23))


In [21]:
# 5 - creates actors
matt_damon = Actor("Matt Damon", date(1970, 10, 8))
dwayne_johnson = Actor("Dwayne Johnson", date(1972, 5, 2))
mark_wahlberg = Actor("Mark Wahlberg", date(1971, 6, 5))


In [22]:
# 6 - add actors to movies
bourne_identity.actors = [matt_damon]
furious_7.actors = [dwayne_johnson]
pain_and_gain.actors = [dwayne_johnson, mark_wahlberg]


In [23]:
# 7 - add contact details to actors
matt_contact = ContactDetails("415 555 2671", "Burbank, CA", matt_damon)
dwayne_contact = ContactDetails("423 555 5623", "Glendale, CA", dwayne_johnson)
dwayne_contact_2 = ContactDetails("421 444 2323", "West Hollywood, CA", dwayne_johnson)
mark_contact = ContactDetails("421 333 9428", "Glendale, CA", mark_wahlberg)


In [24]:
# 8 - create stuntmen
matt_stuntman = Stuntman("John Doe", True, matt_damon)
dwayne_stuntman = Stuntman("John Roe", True, dwayne_johnson)
mark_stuntman = Stuntman("Richard Roe", True, mark_wahlberg)


In [25]:
# 9 - persists data
session.add(bourne_identity)
session.add(furious_7)
session.add(pain_and_gain)

session.add(matt_contact)
session.add(dwayne_contact)
session.add(dwayne_contact_2)
session.add(mark_contact)

session.add(matt_stuntman)
session.add(dwayne_stuntman)
session.add(mark_stuntman)


In [26]:
# display data
# results = connection.execute(db.select([emp])).fetchall()
# df = pd.DataFrame(results)
# df.columns = results[0].keys()
# df.head(4)

In [27]:
# 10 - commit and close session
session.commit()
# session.close()